In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import plotly.io as pio
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
pio.renderers.default = "iframe_connected"

In [2]:
cities_list = ["Mont Saint Michel","St Malo","Bayeux","Le Havre","Rouen","Paris","Amiens","Lille","Strasbourg","Chateau du Haut Koenigsbourg","Colmar","Eguisheim","Besancon","Dijon","Annecy","Grenoble","Lyon","Gorges du Verdon","Bormes les Mimosas","Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"];

In [3]:
cities_list

['Mont Saint Michel',
 'St Malo',
 'Bayeux',
 'Le Havre',
 'Rouen',
 'Paris',
 'Amiens',
 'Lille',
 'Strasbourg',
 'Chateau du Haut Koenigsbourg',
 'Colmar',
 'Eguisheim',
 'Besancon',
 'Dijon',
 'Annecy',
 'Grenoble',
 'Lyon',
 'Gorges du Verdon',
 'Bormes les Mimosas',
 'Cassis',
 'Marseille',
 'Aix en Provence',
 'Avignon',
 'Uzes',
 'Nimes',
 'Aigues Mortes',
 'Saintes Maries de la mer',
 'Collioure',
 'Carcassonne',
 'Ariege',
 'Toulouse',
 'Montauban',
 'Biarritz',
 'Bayonne',
 'La Rochelle']

In [4]:
cities = pd.DataFrame(data = cities_list,columns =['city'])
cities["lat"] = 0
cities["lon"] = 0
cities["rain"] = 0
cities["rain_prob"] = 0

In [5]:
for index,city in cities['city'].iteritems():
    coord = requests.get("https://nominatim.openstreetmap.org/?q=<{},france>&format=json&limit=1".format(city)).json()[0]
    cities.loc[index,"lat"] = coord['lat']
    cities.loc[index,"lon"] = coord['lon']

cities["lat"] = cities['lat'].astype('float')
cities["lon"] = cities['lon'].astype('float')

In [6]:
# Weather
API_key= '116c6fd579d21074a961ae64785fefbd'
weather = requests.get("https://api.openweathermap.org/data/2.5/onecall?lat=48.8566969&lon=2.3514616&exclude=hourly,minutely&appid=116c6fd579d21074a961ae64785fefbd")
days =weather.json()['daily']
expected_rain = [day["pop"]*day["rain"] if 'rain' in day.keys() else 0 for day in days]

In [7]:
API_key = '116c6fd579d21074a961ae64785fefbd'
for index,city in cities['city'].iteritems():
    weather_daily = requests.get("https://api.openweathermap.org/data/2.5/onecall?lat={}&lon={}&exclude=hourly,minutely&appid={}&units=metric".format(cities.loc[index,"lat"],cities.loc[index,"lon"],API_key)).json()['daily']
    daily_rains = [day["pop"]*day["rain"] if 'rain' in day.keys() else 0 for day in weather_daily]
    daily_clouds = [day["clouds"] for day in weather_daily]
    daily_temps = [day["temp"]['max'] for day in weather_daily]
    
    if len(daily_rains) !=0 and len(daily_rains) is not None:
        rain_avg = sum(daily_rains)/len(daily_rains)
    else:
        rain_avg = 0
        
    if len(daily_clouds) !=0 and len(daily_clouds) is not None:
        cloud_avg = sum(daily_clouds)/len(daily_clouds)
    else:
        cloud_avg = 0
    
    if len(daily_temps) !=0 and len(daily_temps) is not None:
        temp_avg = sum(daily_temps)/len(daily_temps)
    else:
        temp_avg = 0
        
    cities.loc[index,"rain"] = rain_avg
    cities.loc[index,"cloudiness"] = cloud_avg
    cities.loc[index,"temperature_max"] = temp_avg
    
    

In [8]:
cities_sorted = cities.sort_values(by='temperature_max',ascending=False,ignore_index=True).loc[:20,:]

In [9]:
fig = px.scatter_mapbox(cities_sorted,hover_name = 'city',lat="lat", lon="lon", color="temperature_max",size="rain", zoom=4, mapbox_style="carto-positron")
fig.show()

In [10]:
# Import os => Library used to easily manipulate operating systems
## More info => https://docs.python.org/3/library/os.html
import os 

# Import logging => Library used for logs manipulation 
## More info => https://docs.python.org/3/library/logging.html
import logging

# Import scrapy and scrapy.crawler 
import scrapy
from scrapy.crawler import CrawlerProcess

ModuleNotFoundError: No module named 'scrapy'

In [ ]:


class bookingSpider(scrapy.Spider):
    
    name ="booking"
    
    start_urls = ["https://www.booking.com/searchresults.fr.html?ss={}".format(city) for city in cities_list]

    def parse(self,response):
        
        hotels = response.xpath("/html/body/div[2]/div[2]/div[2]")
        city = response.request.url.split("ss=")[1].replace("%20",' ')
        
        for hotel in hotels:            
            yield {
                "city" : city,
                "title": hotel.xpath("div/h3/a/text()").getall(),
                "url": hotel.xpath("div/h3/a//@href").getall(),
                "description": hotel.xpath("div/div[1]/p[2]/text()").getall(),
                "scores": hotel.xpath("div/div[1]/div[1]/b").getall()
                  }

filename = "booking.json"

if filename in os.listdir():
        os.remove(filename)

# Declare a new CrawlerProcess with some settings
process = CrawlerProcess(settings = {
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        filename : {"format": "json"},
    }
})

# Start the crawling using the spider you defined above
process.crawl(bookingSpider)
process.start()

In [ ]:
import pandas as pd
#import du fichier dans un dataframe
df =pd.read_json('booking.json')

#isolement de la colonne description
descr = df['description']
df2=df.drop(columns='description')

#traitement (unnest) de description
descr_unested = descr.explode()
descr_clean = descr_unested[descr_unested !='\n']

#unnest le reste des données
df_clean = df2.set_index('city').apply(pd.Series.explode).reset_index()

#concaténation
descr_clean = descr_clean.reset_index(drop=True)
df_final = pd.concat([df_clean,descr_clean],axis=1)

#Nettoyage des données

#complétion de l'url
url_add = pd.Series(["https://www.booking.com" for i in range(df_final.shape[0])])

df_final["url"] = url_add.str.cat(df_final["url"])
df_final["url"] = df_final["url"].str.split("?").apply(lambda x: x[0])

#Récupération des scores
df_final["scores"] = df_final["scores"].str.split('alt="').apply(lambda x: x[1].split('/')[0] if len(x) ==2 else 0)

#Description
df_final["description"] = df_final["description"].str.replace("\n","")

In [ ]:
df_final

In [ ]:
!pip install Scrapy

In [ ]:
import os 

# Import logging => Library used for logs manipulation 
## More info => https://docs.python.org/3/library/logging.html
import logging

# Import scrapy and scrapy.crawler 
import scrapy
from scrapy.crawler import CrawlerProcess

class booking_hotelSpider(scrapy.Spider):
    
    name ="hotel_page"
    
    start_urls = [url for url in df_final["url"]]

    def parse(self,response):
          
        yield {
                "coords": response.xpath("/html/body/div[2]/div[2]/div[1]/div[2]/p/a//@data-coords").getall()
                  }

filename = "hotel_page.json"

if filename in os.listdir():
        os.remove(filename)

# Declare a new CrawlerProcess with some settings
process = CrawlerProcess(settings = {
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        filename : {"format": "json"},
    }
})

# Start the crawling using the spider you defined above
process.crawl(booking_hotelSpider)
process.start()

In [ ]:
coords =pd.read_json('hotel_page.json')
df_final['lon'] = coords['coords'].apply(lambda x: x[0].split(",")[0]).astype('float')
df_final['lat'] = coords['coords'].apply(lambda x: x[0].split(",")[1]).astype('float')
df_final['scores'] = df_final['scores'].astype('float')

In [ ]:
df_final.dtypes

In [ ]:
fig = px.scatter_mapbox(df_final,hover_name = 'title',lat="lat",color='scores', lon="lon",size="scores", zoom=4, mapbox_style="carto-positron")
fig.show()